### Networks ZOO :: Seerslab inc.

- ShuffleNet 

- EffiecientNet 
    - Scaling factor 

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

In [ ]:
def EffiecientNet():
    

In [ ]:
model = tf.keras.applications.ResNet50()

In [ ]:
model.summary()

In [1]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.
        x = self.d1(x)

    return self.d2(x)

model = MyModel()